In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import scipy

In [2]:
products_df_preview = pd.read_csv('en.openfoodfacts.org.products.csv', delimiter='\t',
                          encoding='utf-8', low_memory=False, nrows=10,
                          dtype={"code":"string"})
with pd.option_context('display.max_columns', None):
    display(products_df_preview[['code', 'product_name', 'brands', 'categories', 'categories_tags', 'categories_en', 'ingredients_text',
                         'ingredients_tags', 'pnns_groups_1', 'pnns_groups_2', 'food_groups', 'food_groups_tags', 'food_groups_en',
                         'main_category', 'main_category_en', 'energy-kj_100g', 'energy-kcal_100g', 'energy_100g', 'fat_100g',
                         'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'alcohol_100g',
                         'glycemic-index_100g']])

,code,product_name,brands,categories,categories_tags,categories_en,ingredients_text,ingredients_tags,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,main_category,main_category_en,energy-kj_100g,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,alcohol_100g,glycemic-index_100g
0,00000000000000225,jeunes pousses,endives,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000000207025004,Andrè,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,165.0,690.0,2.0,2.00,65.0,12.60,3.0,1.5,NaN,NaN
2,00000000000003429145,L.casei,NaN,NaN,NaN,NaN,"Leche semidesnatada, azucar 6.9%, leche desnat...","en:semi-skimmed-milk,en:dairy,en:milk,en:sugar...",unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.4,0.90,9.8,9.80,NaN,2.7,NaN,NaN
3,00000000000026772226,Skyr,Danone,"Produits laitiers,Desserts,Produits fermentés,...","en:dairies,en:desserts,en:fermented-foods,en:f...","Dairies,Desserts,Fermented foods,Fermented mil...",NaN,NaN,Milk and dairy products,Dairy desserts,en:dairy-desserts,"en:milk-and-dairy-products,en:dairy-desserts","Milk and dairy products,Dairy desserts",en:skyr,Skyr,NaN,57.0,238.0,0.2,0.10,3.9,3.90,NaN,10.0,NaN,NaN
4,0000000000017,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,375.0,1569.0,7.0,3.08,70.1,15.00,NaN,7.8,NaN,NaN
5,0000000000031,Cacao,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,000000000003327986,Filetes de pollo empanado,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN,685.8,163.9,685.8,1.9,1.00,NaN,NaN,NaN,15.3,NaN,NaN
7,000000000004128579,Burger Meat Pollo,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,194.0,812.0,11.0,3.90,5.7,0.05,NaN,18.0,NaN,NaN
8,000000000004622327,Hamburguesas de ternera 100%,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,874.9,3661.0,15.1,6.10,2.6,1.00,NaN,15.7,NaN,NaN
9,00000000001,hyde icon,NaN,NaN,NaN,NaN,"beta alanine, creatine hcl, ancient peat & app...","en:beta-alanine,en:creatine-hcl,en:ancient-pea...",unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
for index, row in products_df_preview.iterrows():
    kj = row['energy-kj_100g']
    kj_to_kcal = 0      
    if np.isnan(kj):
        kj = row['energy_100g']
    if np.isnan(kj):
        kj = 0
    else:
        kj_to_kcal = kj * 0.239006
    kcal = row['energy-kcal_100g']
    if np.isnan(kcal):
        kcal = 0
    kc = row['energy-kcal_100g']
    fat = row['fat_100g']
    carbs = row['carbohydrates_100g']
    fiber = row['fiber_100g']
    protein = row['proteins_100g']
    alcohol = row['alcohol_100g']
    fat_cal = 0
    carb_cal = 0
    fiber_cal = 0
    prot_cal = 0
    alc_cal = 0
    missing = ""
    if not np.isnan(fat):
        fat_cal = 9 * fat
    else:
        missing += "Fat "
    if not np.isnan(carbs):
        carb_cal = 4 * carbs
    else:
        missing += "Carbs "
    if not np.isnan(fiber):
        fiber_cal = 2 * carbs
    else:
        missing += "Fiber "
    if not np.isnan(protein):
        prot_cal = 4 * protein
    else:
        missing += "Protein "
    if not np.isnan(alcohol):
        alc_cal = 7 * alcohol
    else:
        missing += "Alcohol "
    estim_kcal = fat_cal + carb_cal + fiber_cal + prot_cal + alc_cal
    print(f"Stated kJ: {kj}\nkJ to kcal: {kj_to_kcal}\nStated kcal: {kcal}\nEstimated kcal: {estim_kcal}\nMissing: {missing}\n")

Stated kJ: 0
kJ to kcal: 0
Stated kcal: 0
Estimated kcal: 0
Missing: Fat Carbs Fiber Protein Alcohol 

Stated kJ: 690.0
kJ to kcal: 164.91414
Stated kcal: 165.0
Estimated kcal: 414.0
Missing: Alcohol 

Stated kJ: 0
kJ to kcal: 0
Stated kcal: 0
Estimated kcal: 62.60000000000001
Missing: Fiber Alcohol 

Stated kJ: 238.0
kJ to kcal: 56.883428
Stated kcal: 57.0
Estimated kcal: 57.4
Missing: Fiber Alcohol 

Stated kJ: 1569.0
kJ to kcal: 375.000414
Stated kcal: 375.0
Estimated kcal: 374.59999999999997
Missing: Fiber Alcohol 

Stated kJ: 0
kJ to kcal: 0
Stated kcal: 0
Estimated kcal: 0
Missing: Fat Carbs Fiber Protein Alcohol 

Stated kJ: 685.8
kJ to kcal: 163.91031479999998
Stated kcal: 163.9
Estimated kcal: 78.3
Missing: Carbs Fiber Alcohol 

Stated kJ: 812.0
kJ to kcal: 194.072872
Stated kcal: 194.0
Estimated kcal: 193.8
Missing: Fiber Alcohol 

Stated kJ: 3661.0
kJ to kcal: 875.000966
Stated kcal: 874.9
Estimated kcal: 209.10000000000002
Missing: Fiber Alcohol 

Stated kJ: 0
kJ to kcal: 0

In [4]:
products_df = pd.DataFrame()

if not os.path.isfile('products_df.ftr'):
    chunksize = 10 ** 5
    with pd.read_csv('en.openfoodfacts.org.products.csv', delimiter='\t', encoding='utf-8', low_memory=False, chunksize=chunksize,
                    dtype={"code":"string"}) as reader:
        for chunk in reader:
            chunk.dropna(subset=['categories_en', 'fat_100g', 'carbohydrates_100g', 'proteins_100g'], inplace=True)
            chunk.dropna(subset=['energy-kj_100g', 'energy-kcal_100g', 'energy_100g'], how='all', inplace=True)
            chunk = chunk[['code', 'product_name', 'brands', 'categories', 'categories_tags', 'categories_en', 'ingredients_text',
                        'ingredients_tags', 'pnns_groups_1', 'pnns_groups_2', 'food_groups', 'food_groups_tags', 'food_groups_en',
                        'main_category', 'main_category_en', 'energy-kj_100g', 'energy-kcal_100g', 'energy_100g', 'fat_100g',
                        'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'alcohol_100g',
                        'glycemic-index_100g']]
            products_df = pd.concat([products_df, chunk])
    products_df.reset_index(drop=True, inplace=True)
    products_df.to_feather('products_df.ftr')
else:
    products_df = pd.read_feather('products_df.ftr')


In [5]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904870 entries, 0 to 904869
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   code                 904870 non-null  string 
 1   product_name         901503 non-null  object 
 2   brands               645608 non-null  object 
 3   categories           904870 non-null  object 
 4   categories_tags      904870 non-null  object 
 5   categories_en        904870 non-null  object 
 6   ingredients_text     604849 non-null  object 
 7   ingredients_tags     604512 non-null  object 
 8   pnns_groups_1        904870 non-null  object 
 9   pnns_groups_2        904870 non-null  object 
 10  food_groups          743353 non-null  object 
 11  food_groups_tags     743353 non-null  object 
 12  food_groups_en       743353 non-null  object 
 13  main_category        904870 non-null  object 
 14  main_category_en     904870 non-null  object 
 15  energy-kj_100g   

In [6]:
products_df.drop('glycemic-index_100g', axis=1, inplace=True)